In [37]:
import torch
import numpy as np

In [38]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

np_array = np.array(data)
x_np = torch.from_numpy(np_array)

x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.4048, 0.9703],
        [0.0964, 0.5917]]) 



In [39]:
#Tensor Attributes
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


In [40]:
if torch.cuda.is_available():
    tensor = tensor.to('cuda')
print(f"Device tensor is stored on: {tensor.device}")

Device tensor is stored on: cpu


In [41]:
tensor = torch.ones(4, 4)
tensor[:,1] = 0
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


tensor 之间的运算

In [42]:
#Multiplying tensors
print(f"tensor.mul(tensor) \n{tensor.mul(tensor)} \n")  # This computes the element-wise product
print(f"tensor * tensor \n{tensor * tensor}")  # Alternative syntax:

tensor.mul(tensor) 
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor * tensor 
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [43]:
print(f"tensor.matmul(tensor.T) \n {tensor.matmul(tensor.T)} \n") # Alternative syntax:
print(f"tensor @ tensor.T \n {tensor @ tensor.T}")

tensor.matmul(tensor.T) 
 tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]]) 

tensor @ tensor.T 
 tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])


In-place operations Operations that have a _ suffix are in-place. For example: x.copy_(y), x.t_(), will change x.

In [44]:
tensor = torch.ones(4, 4)
print(tensor, "\n")
#tensor.add_(5)
tensor.mul_(5)
print(tensor)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]) 

tensor([[5., 5., 5., 5.],
        [5., 5., 5., 5.],
        [5., 5., 5., 5.],
        [5., 5., 5., 5.]])


Attention! Tensors on the CPU and NumPy arrays can share their underlying memory locations, and changing one will change the other.

In [45]:
t = torch.ones(4, 4)
n = t.numpy()
#print(f"t: \n{t}")
#print(f"n: \n{n}")

n[:, 1] = 0
print(f"t: \n{t}")

t: 
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [46]:
n = np.ones([3, 3])
t = torch.from_numpy(n)
n2 = t.numpy()
print(n2)

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
